# Benchmarking Test File

Use this notebook to compare a run of SPECTRA with the benchmark data. 

The data you're comparing is in the folder marked "benchmark_factors" on the google drive. There is one csv for each factor, and each file is named the same as the factor you should be comparing it against.

There are two main things we're looking at - pairwise correlation between each factor and the intersection of the genes that contribute to each factor.

In [2]:
# imports
import re
import scanpy as sc
import seaborn
import numpy as np
import csv
import pandas as pd

This is all buggy right now - mainly waiting to see how to match factor weights back to genes, and waiting to see if a new run returns each factor individually like this, or if we have to pull that apart ourselves.

In [ ]:
# Importing files
# file path for benchmark data - remember to CHANGE THE FILE PATH TO YOUR LOCAL PLACE YOU WANT THESE TO BE
benchmark_path = "C:\\Users\\phill\\Documents\\HMCFall22\\Clinic\\"
# file path for new data - remember to CHANGE THE FILE PATH TO YOUR LOCAL PLACE YOU WANT THESE TO BE
new_factor_path = 'C:/Users/phill/Documents/HMCFall22/Clinic/SPECTRA-GPU/new_factors/'

# new run - remember to CHANGE THE FILE PATH TO YOUR LOCAL PLACE YOU WANT THESE TO BE
adata = sc.read_h5ad(benchmark_path + 'data_for_clinic_2023.h5ad')

# grabbing the list of factors from the new data
def match(input_string):
    return re.findall(r'[\d]*-X-[\w_-]*X-norm', input_string)
    
factor_matches = [match(word) for word in adata.obs]

# the list of the factor names as strings
factor_list = []

for l in factor_matches:
    if l:
        factor_list.append(l[0])

# put this into a csv - CHANGE THE FILE PATH TO YOUR LOCAL PLACE YOU WANT THESE TO BE
for factor in factor_list:
    newFactors = pd.DataFrame(adata.obs[factor]).to_csv(new_factor_path + f"{factor}.csv", header=False, index=False)



Needs to be done below
- be clear on dimensions of factor lists
- right now this is working with GENE SCORE NOT FACTOR GENE NAMES
- i tagged on "genes" to the end of the file path even though i'm not sure that's how that will work. just a note to myself, as it were

In [1]:
# Intersection between factors
# for now assuming the anndata looks identical to the benchmark set we recieved.

for l in factor_list:
    benchmark_factor = []
    new_factor = []
    
    # grab the benchmark data
    with open(benchmark_path + '{l}_genes.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # get list of factors from csv
        benchmark_factor = list(csv_reader)

        csv_file.close()

    # get the new run's data
    with open(new_factor_path + '{l}_genes.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # get list of factors from csv
        new_factor = list(csv_reader)

        csv_file.close()



    benchmark_factor_set = set(benchmark_factor)
    new_factor_set = set(new_factor)

    intersection_1 = benchmark_factor_set.intersection(new_factor_set) 
    average_unique_genes = (len(benchmark_factor_set) + len(new_factor_set))//2  

    print("Intersection between genes for factor {l[0]}: " + str(intersection_1))
    print("Percent error of intersection: " + str(round(abs(intersection_1 - average_unique_genes)/average_unique_genes, 4) * 100) + "%\n\n")


IndentationError: expected an indented block (3081531705.py, line 5)

In [ ]:
# Pairwise correlation 

for l in factor_list:
    benchmark_factor = []
    new_factor = []
    
    # grab the benchmark data
    with open(benchmark_path + '{l}.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # get list of factors from csv
        benchmark_factor = list(csv_reader)

        csv_file.close()

    # convert benchmark_factor elements to floats
    num_benchmark_factor = []

    for r in range(len(benchmark_factor)):
        num_benchmark_factor.append(float(benchmark_factor[r][0]))

    # get the new run's data
    with open(new_factor_path + '{l}.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # get list of factors from csv
        new_factor = list(csv_reader)

        csv_file.close()
    
    # convert new_factor elements to floats
    num_new_factor = []

    for r in range(len(new_factor)):
        num_new_factor.append(float(new_factor[r][0]))

    #for r in range(len(new_factor)):
    #    row = []
    #    for c in range(len(new_factor[r])):
    #        row.append(float(new_factor[r][c]))
    #    num_new_factor.append(row)

    corr_mat = np.corrcoef(num_benchmark_factor, num_new_factor)

    seaborn.clustermap(corr_mat)